---
topic: "Topic 6A: Machine Learning/AI"
subtitle: "UAS Change Detection"
title: "Assignment 6B"
original-assignment-due-date: 11/06/2024
assignment-due-date: 12/1/2025
author:
    - Corey White
date: 11/10/25
format: html
toc-depth: 4
activity: Assignment/Lab
keep-ipynb: true
notebook-view:
  - notebook: Assignment6B.ipynb
    title: "UAS Image Classification"
    url: https://colab.research.google.com/drive/1sia6F1oPW1ISV8a-6YkEEhJhXriVrYdG?usp=sharing
jupyter: python3
execute: 
  eval: false
  enabled: true
  freeze: auto
  output: true
draft: true
---

## Objective
Apply DEM of Difference and NDVI differencing to provided datasets 

Focus on:

- Co-registration and alignment accuracy
- Thresholding and LoD95 calculation
- Quantifying and visualizing uncertainty

In [1]:
# Import standard Python packages we need
import sys
import subprocess
from pathlib import Path

# Ask GRASS where its Python packages are to be able to run it from the notebook
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

In [2]:
# Import the GRASS packages
import grass.script as gs
import grass.jupyter as gj

In [3]:
# Start GRASS in default project mapset
home_directory = Path.home()
grassdata = Path(home_directory, "grassdata", "northcarolina_coast_spm")
session = gj.init(grassdata)

In [4]:
layers = gs.parse_command("g.list", type="all", format="json")

for i in layers:
    print(i)

In [5]:
gs.run_command("g.region", region="jrdunes_1m", flags="p")

In [6]:
m = gj.Map(filename="./../images/airphoto_1932.png")
m.d_rast(map="airphoto_1932")
m.d_barscale()
m.show()

m = gj.Map(filename="./../images/airphoto_1932_hist.png")
m.d_histogram(map="airphoto_1932")
m.show()

m = gj.Map(filename="./../images/airphoto_1945.png")
m.d_rast(map="airphoto_1945")
m.d_barscale()
m.show()

m = gj.Map(filename="./../images/airphoto_1955.png")
m.d_rast(map="airphoto_1955")
m.d_barscale()
m.show()

m = gj.Map(filename="./../images/airphoto_1962.png")
m.d_rast(map="airphoto_1962")
m.d_barscale()
m.show()

m = gj.Map(filename="./../images/airphoto_1974.png")
m.d_rast(map="airphoto_1974")
m.d_barscale()
m.show()

m = gj.Map(filename="./../images/airphoto_1988.png")
m.d_rast(map="airphoto_1988")
m.d_barscale()
m.show()

m = gj.Map(filename="./../images/airphoto_2007.png")
m.d_rast(map="airphoto_2007")
m.d_barscale()
m.show()

m = gj.Map(filename="./../images/airphoto_2009.png")
m.d_rast(map="airphoto_2009")
m.d_barscale()
m.show()

m = gj.Map(filename="./../images/airphoto_2009_hist.png")
m.d_histogram(map="airphoto_2009")
m.show()

In [7]:
gs.run_command("r.info", map="airphoto_2009")

gs.run_command("r.rgb", input="airphoto_2009", red="airphoto_2009.r", green="airphoto_2009.g", blue="airphoto_2009.b")
m = gj.Map(filename="./../images/airphoto_2009_r_hist.png")
m.d_histogram(map="airphoto_2009.r")
m.show()

In [8]:
# airphotos = [1932, 1945, 1955, 1962, 1974, 1988, 2007, 2009]

gs.run_command("t.create",
    type="strds",
    temporaltype="absolute",
    output="jr_airphotos",
    title="Jockey's Ridge Air Photos",
    description="Jockey's Ridge Air Photos from 1932 - 2009"
)
airphotos_list = gs.parse_command("g.list", type="raster", pattern="airphoto_*", format="json")
airphotos = [i.get('fullname') for i in airphotos_list]
gs.run_command(
    "t.register",
    type="raster",
    input="jr_airphotos",
    file="./resources/airphotos.txt"
)

In [9]:
# gs.run_command("t.connect", flags="p")
# gs.run_command("t.info", input="jr_airphotos")
import sqlite3
con = sqlite3.connect(Path(home_directory, "grassdata", "northcarolina_coast_spm", "PERMANENT", "tgis", "sqlite.db"))
cur = con.cursor()
# res = cur.execute("SELECT * FROM sqlite_master WHERE table = 'tgis_metadata'")
for row in cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"):
    print(row)

# row = res.fetchone()
# print(row)
con.close()

In [10]:
m = gj.TimeSeriesMap()
m.add_raster_series("jr_airphotos", fill_gaps=False)
m.d_barscale()
m.show()